Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# DataTables, Indexes, Pandas, and Seaborn

## Some useful (free) resources

Introductory:

* [Getting started with Python for research](https://github.com/TiesdeKok/LearnPythonforResearch), a gentle introduction to Python in data-intensive research.

* [A Whirlwind Tour of Python](https://jakevdp.github.io/WhirlwindTourOfPython/index.html), by Jake VanderPlas, another quick Python intro (with notebooks).

Core Pandas/Data Science books:

* [The Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/), by Jake VanderPlas.

* [Python for Data Analysis, 2nd Edition](http://proquest.safaribooksonline.com/book/programming/python/9781491957653), by  Wes McKinney, creator of Pandas. [Companion Notebooks](https://github.com/wesm/pydata-book)

* [Effective Pandas](https://github.com/TomAugspurger/effective-pandas), a book by Tom Augspurger, core Pandas developer.


Complementary resources:

* [An introduction to "Data Science"](https://github.com/stefanv/ds_intro), a collection of Notebooks by BIDS' [Stéfan Van der Walt](https://bids.berkeley.edu/people/st%C3%A9fan-van-der-walt).

* [Effective Computation in Physics](http://proquest.safaribooksonline.com/book/physics/9781491901564), by Kathryn D. Huff; Anthony Scopatz. [Notebooks to accompany the book](https://github.com/physics-codes/seminar). Don't be fooled by the title, it's a great book on modern computational practices with very little that's physics-specific.


OK, let's load and configure some of our core libraries (as an aside, you can find a nice visual gallery of available matplotlib sytles [here](https://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html)).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

plt.style.use('fivethirtyeight')
sns.set_context("notebook")

## Getting the Data

https://www.ssa.gov/OACT/babynames/index.html

https://www.ssa.gov/data

As we saw before, we can download data from the internet with Python, and do so only if needed:

In [ ]:
import requests
from pathlib import Path

namesbystate_path = Path('namesbystate.zip')
data_url = 'https://www.ssa.gov/oact/babynames/state/namesbystate.zip'

if not namesbystate_path.exists():
    print('Downloading...', end=' ')
    resp = requests.get(data_url)
    with namesbystate_path.open('wb') as f:
        f.write(resp.content)
    print('Done!')

Let's use Python to understand how this data is laid out:

In [ ]:
import zipfile
zf = zipfile.ZipFile(namesbystate_path, 'r')
print([f.filename for f in zf.filelist])

We can pull the PDF readme to view it, but let's operate with the rest of the data in its compressed state:

In [ ]:
zf.extract('StateReadMe.pdf')

Let's have a look at the California data, it should give us an idea about the structure of the whole thing:

In [ ]:
ca_name = 'CA.TXT'
with zf.open(ca_name) as f:
    for i in range(10):
        print(f.readline().rstrip().decode())

This is equivalent (on macOS or Linux) to extracting the full `CA.TXT` file to disk and then using the `head` command (if you're on Windows, don't try to run the cell below):

In [ ]:
zf.extract(ca_name)
!head {ca_name}

In [ ]:
!cat /tmp/environment.yml

In [ ]:
!echo {ca_name}

A couple of practical comments:

* The above is using special tricks in IPython that let you call operating system commands via `!cmd`, and that expand Python variables in such commands with the `{var}` syntax. You can find more about IPython's special tricks [in this tutorial](https://github.com/ipython/ipython-in-depth/blob/master/examples/IPython%20Kernel/Beyond%20Plain%20Python.ipynb).

* `head` doesn't work on Windows, though there are equivalent Windows commands. But by using Python code, even if it's a little bit more verbose, we have a 100% portable solution.

* If the `CA.TXT` file was huge, it would be wasteful to write it all to disk only to look at the start of the file.

The last point is an important, and general theme of this course: we need to learn how to operate with data only on an as-needed basis, because there are many situations in the real world where we can't afford to brute-force 'download all the things'.

Let's remove the `CA.TXT` file to make sure we keep working with our compressed data, as if we couldn't extract it:

In [ ]:
import os; os.unlink(ca_name)

## Question 1: What was the most popular name in CA last year?

In [ ]:
import pandas as pd

field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    ca = pd.read_csv(fh, header=None, names=field_names)
ca.head()

### Indexing Review

Let's play around a bit with our indexing techniques from earlier today.

In [ ]:
ca['Count'].head()

In [ ]:
ca[0:3]

In [ ]:
#ca[0]

In [ ]:
ca.iloc[:3, -2:]

In [ ]:
ca.loc[0:3, 'State']

In [ ]:
ca['Name'].head()

In [ ]:
ca[['Name']].head()

In [ ]:
ca[ca['Year'] == 2017].tail()

## Understanding the Data

In [ ]:
ca.head()

We can get a sense for the shape of our data:

In [ ]:
ca.shape

In [ ]:
ca.size  # rows x columns

Pandas will give us a summary overview of the *numerical* data in the DataFrame:

In [ ]:
ca.describe()

And let's look at the *structure* of the DataFrame:

In [ ]:
ca.index

### Sorting

What we've done so far is NOT exploratory data analysis. We were just playing around a bit with the capabilities of the pandas library. Now that we're done, let's turn to the problem at hand: Identifying the most common name in California last year.

In [ ]:
ca2017 = ca[ca['Year'] == 2017]
ca_sorted = ca2017.sort_values('Count', ascending=False).head(10)
ca_sorted

## Question 2: Most popular names in all states for each year of each gender?

### Put all DFs together

Again, we'll work off our in-memory, compressed zip archive and pull the data out of it into Pandas DataFrames without ever putting it all on disk. We can see how large the compressed and uncompressed data is:

In [ ]:
sum(f.file_size for f in zf.filelist)/1_000_000

In [ ]:
sum(f.compress_size for f in zf.filelist)/1_000_000

In [ ]:
__/_  # divide the next-previous result by the previous one

First, notice that the in-memory compressed archive is not alphabetically sorted:

In [ ]:
zf.filelist[:5]

But we can pull them out in alphabetical order with the Python `sorted` function, sorting by filename:

In [ ]:
%%time
states = []

for f in sorted(zf.filelist, key=lambda x:x.filename):
    if not f.filename.endswith('.TXT'):
        continue
    with zf.open(f) as fh:
        states.append(pd.read_csv(fh, header=None, names=field_names))

Now, we create a single DataFrame by concatenating these into one:

In [ ]:
baby_names = pd.concat(states).reset_index(drop=True)
baby_names.head()

In [ ]:
baby_names.shape

### Group by state and year

In [ ]:
baby_names[
    (baby_names['State'] == 'CA')
    & (baby_names['Year'] == 1995)
    & (baby_names['Sex'] == 'M')
].head()

# Now I could write 3 nested for loops...

In [ ]:
%%time
baby_names.groupby('State').size().head()

In [ ]:
state_counts = baby_names.loc[:, ('State', 'Count')]
state_counts.head()

In [ ]:
sg = state_counts.groupby('State')
sg

In [ ]:
state_counts.groupby('State').sum().head()

For Data 8 veterans, this is equivalent to this code from Data 8:

    state_and_groups.group('State', np.sum)
    
In pandas, could also use agg here, yielding:

    state_counts.groupby('State').agg(np.sum)

### Grouping by multiple columns

In [ ]:
baby_names.groupby(['State', 'Year']).size().head(3)

In [ ]:
baby_names.groupby(['State', 'Year']).sum().head(3)

In [ ]:
baby_names.groupby(['State', 'Year', 'Sex']).sum().head()

In [ ]:
%%time
def first(series):
    '''Returns the first value in the series.'''
    return series.iloc[0]

most_popular_names = baby_names.groupby(['State', 'Year', 'Sex']).agg(first)

most_popular_names.head()

This creates a multilevel index. It is quite complex, but just know that you can still slice:

In [ ]:
most_popular_names[most_popular_names['Name'] == 'Samuel']

And you can use `.loc` as so:

In [ ]:
most_popular_names.loc['CA', 1997, 'M']

## Submission

You're done!

Before submitting this assignment, ensure to:

1. Restart the Kernel (in the menubar, select Kernel->Restart & Run All)
2. Validate the notebook by clicking the "Validate" button

Finally, make sure to **submit** the assignment via the Assignments tab in Datahub